In [4]:
%load_ext autoreload

%autoreload 2
import time

import pandas
import argparse

import chainer
import chainer.functions as F
import chainer.links as L
from chainer import training
from chainer.training import extensions


from chainer.datasets import tuple_dataset
from chainer.dataset import convert, concat_examples

import numpy
import os
import glob
import sys
from pathlib import Path


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
class MLP(chainer.Chain):

    def __init__(self, n_units, n_out):
        super(MLP, self).__init__()
        with self.init_scope():
            # the size of the inputs to each layer will be inferred
            self.l1 = L.Linear(None, n_units)  # n_in -> n_units
            self.l2 = L.Linear(None, n_units)  # n_units -> n_units
            self.l3 = L.Linear(None, n_out)  # n_units -> n_out

    def __call__(self, x):
        h1 = F.relu(self.l1(x))
        h2 = F.relu(self.l2(h1))
        return self.l3(h2)

In [6]:
import easy_chainer

In [4]:
front_list = sorted(glob.glob("C:/Users/Owner/Desktop/LAB/LAB 2019/analysis/mix_fabric/CP*F*"))
back_list = sorted(glob.glob("C:/Users/Owner/Desktop/LAB/LAB 2019/analysis/mix_fabric/CP*B*"))

In [5]:
f = open("C:/Users/Owner/Desktop/LAB/LAB 2019/analysis/mix_fabric/CP01B1.txt","r")

In [6]:
cp_list = sorted(glob.glob("C:/Users/Owner/Desktop/LAB/LAB 2019/analysis/mix_fabric/CP*.txt"))


In [5]:
xls = pandas.read_csv(front_list[0], skiprows=4, delimiter="\t", header=None)
col = xls.as_matrix()[:,0]

j = 0
for list_ in (front_list, back_list):
#     print(list_)
    for i, fname in enumerate (list_):
        xls = pandas.read_csv(fname, comment="#", delimiter="\t", header=None)
        if j == 0:
            arr = xls.as_matrix()[:,1]
            hddr = os.path.split(fname)[1].split(".")[0]
        else:
            arr = numpy.vstack((arr, xls.as_matrix()[:,1]))
            hddr = numpy.hstack((hddr, os.path.split(fname)[1].split(".")[0]))
        j += 1

C:\Users\Owner\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  
C:\Users\Owner\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  # Remove the CWD from sys.path while we load stuff.
C:\Users\Owner\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  del sys.path[0]


In [6]:
print(arr.shape, col.shape, hddr.shape)

(1249, 1713) (1713,) (1249,)


In [7]:
front_list[0:1]

['C:/Users/Owner/Desktop/LAB/LAB 2019/analysis/mix_fabric\\CP01F1.txt']

In [8]:
data, teach = easy_chainer.load_Data("C:/Users/Owner/Desktop/LAB/LAB 2019/analysis/mix_fabric/CP.xlsx")
data = data.astype(numpy.float32)
teach = teach
print(teach)

[76.7 65.8 79.6 79.8 81.6 80.9 83.2 83.4 63.  71.1 48.2 53.1 53.8 62.1
 62.3 64.3 66.3 32.1 75.1 78.5 62.8 63.8 83.4 83.9 84.2 84.2 84.4 84.6
 80.7 66.5 85.2 41.2 38.3 31.5 45.2 64.6 55.3 63.  37.6 38.6 50.3 27.9
 27.6 22.3 20.7  1.6 19.9 70.3 20.  20.2 19.5 27.4 47.2 10.9 19.4 65.6
 90.5  9.  66.3 63.7 42.5 66.  20.2 21.2 29.8 42.4 32.3 49.6 50.3 63.4
 63.6 61.3 64.6 64.4 63.9 77.8 65.8 65.6 81.5 63.9 70.4 65.4 65.6 65.6
 62.5 61.2 62.6 62.3 62.9 72.6 62.5 62.4 79.5 82.5 81.9 82.3 81.5 82.4
 82.3 82.8 82.2 82.2 81.9 82.7 83.3 15.3 77.9 82.6 67.5 91.6 91.2 36.
 21.9 66.4 66.  62.  81.2 65.7 68.4 78.1 83.5 79.6 83.1 69.6 65.7  0.4
 83.8]


C:\Users\Owner\Desktop\LAB\LAB 2018\jupyter File\easy_chainer.py:87: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  data = xls.as_matrix()[:-1]
C:\Users\Owner\Desktop\LAB\LAB 2018\jupyter File\easy_chainer.py:88: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  teach = xls.as_matrix()[-1]


In [17]:
teach = teach.astype(numpy.int8)
teach_bins = numpy.histogram(teach, bins=numpy.max(teach) + 1)
teach_bins

(array([ 1,  1,  0,  0,  0,  0,  0,  0,  0,  1,  1,  0,  0,  0,  0,  1,  0,
         0,  0,  3,  4,  2,  1,  0,  0,  0,  0,  3,  0,  1,  0,  1,  2,  0,
         0,  0,  1,  1,  2,  0,  0,  1,  2,  0,  0,  1,  0,  1,  1,  1,  2,
         0,  0,  2,  0,  1,  0,  0,  0,  0,  0,  2, 10,  8,  4,  9,  6,  1,
         1,  1,  2,  1,  1,  0,  0,  1,  1,  2,  2,  4,  2,  6,  9,  8,  4,
         1,  0,  0,  0,  0,  1,  2], dtype=int64),
 array([ 0.        ,  0.98913043,  1.97826087,  2.9673913 ,  3.95652174,
         4.94565217,  5.93478261,  6.92391304,  7.91304348,  8.90217391,
         9.89130435, 10.88043478, 11.86956522, 12.85869565, 13.84782609,
        14.83695652, 15.82608696, 16.81521739, 17.80434783, 18.79347826,
        19.7826087 , 20.77173913, 21.76086957, 22.75      , 23.73913043,
        24.72826087, 25.7173913 , 26.70652174, 27.69565217, 28.68478261,
        29.67391304, 30.66304348, 31.65217391, 32.64130435, 33.63043478,
        34.61956522, 35.60869565, 36.59782609, 37.58695652

In [ ]:
k = 7


for i in range(int(max(teach) + 1)):
    tmp0_arr = numpy.where(teach == i)[0]
    try:
        tmp0 = numpy.random.choice(tmp0_arr, int(k), replace=False)
        if i == 0:
            id_train = tmp0
        else:
            id_train = numpy.hstack((id_train, tmp0))
    except Exception:
        print("Not teach")

In [10]:
import random

In [12]:
teach = teach.astype(numpy.float32)

In [11]:
id_all = numpy.arange(1, len(teach) + 1, 1).astype(numpy.int32) - 1
print(id_all)

[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126]


In [13]:
id_train = numpy.random.choice(id_all, 100, replace=False) #重複なし
print(id_train)

[  5 100  10   8  11  17  48  43  44  28  34  79  61  74  99  93 117 105
  30  92 112   6  51 124 118 111   1  49  83 115  59  72 104  94  23  78
  80  46  24  57  38  55  29  70  85  54  37 109 103  66  58   0  82  67
  12   4  75 125  52  63 123 116  69  76 122 102 107  19  98  90 120  65
 101   7 106 119  81 114  97 113  56  32  14  21  15  87  88  71  41  62
  40  13  84  68 110  77   2  73  89  36]


In [14]:
id_test = numpy.delete(id_all, id_train)
print(id_test)

[  3   9  16  18  20  22  25  26  27  31  33  35  39  42  45  47  50  53
  60  64  86  91  95  96 108 121 126]


In [15]:
x_train, y_train = data[:, id_train], teach[id_train]
x_test, y_test = data[:, id_test], teach[id_test]

In [17]:
y_train

array([80.9, 82.2, 48.2, 63. , 53.1, 32.1, 20. , 22.3, 20.7, 80.7, 45.2,
       63.9, 66. , 63.9, 82.8, 82.5, 65.7, 15.3, 85.2, 79.5, 21.9, 83.2,
       27.4, 65.7, 68.4, 36. , 65.8, 20.2, 65.6, 62. , 63.7, 64.6, 83.3,
       81.9, 83.9, 81.5, 70.4, 19.9, 84.2,  9. , 37.6, 65.6, 66.5, 63.6,
       61.2, 19.4, 63. , 91.6, 82.7, 32.3, 66.3, 76.7, 65.6, 49.6, 53.8,
       81.6, 77.8,  0.4, 47.2, 21.2, 69.6, 81.2, 63.4, 65.8, 83.1, 81.9,
       82.6, 78.5, 82.3, 62.5, 83.5, 42.4, 82.2, 83.4, 77.9, 78.1, 65.4,
       66. , 82.4, 66.4, 90.5, 38.3, 62.3, 63.8, 64.3, 62.3, 62.9, 61.3,
       27.9, 20.2, 50.3, 62.1, 62.5, 50.3, 91.2, 65.6, 79.6, 64.4, 72.6,
       55.3], dtype=float32)

In [18]:
train = tuple_dataset.TupleDataset(x_train.T, y_train.reshape(-1, ))
test = tuple_dataset.TupleDataset(x_test.T, y_test.reshape(-1, ))

train_iter = chainer.iterators.SerialIterator(train, 100)
test_iter = chainer.iterators.SerialIterator(test, 27, repeat=False, shuffle=False)

In [ ]:
train = tuple_dataset.TupleDataset(x_train.T, y_train.reshape(-1, ))
test = tuple_dataset.TupleDataset(x_test.T, y_test.reshape(-1, ))

train_iter = chainer.iterators.SerialIterator(train, 100)
test_iter = chainer.iterators.SerialIterator(test, 27, repeat=False, shuffle=False)

In [19]:
class MLP(chainer.Chain):

    def __init__(self, n_units, n_out):
        super(MLP, self).__init__()
        with self.init_scope():
            # the size of the inputs to each layer will be inferred
            self.l1 = L.Linear(None, n_units)  # n_in -> n_units
            self.l2 = L.Linear(None, n_units)  # n_units -> n_units
            self.l3 = L.Linear(None, n_out)  # n_units -> n_out

    def __call__(self, x):
        h1 = F.relu(self.l1(x))
        h2 = F.relu(self.l2(h1))
        return self.l3(h2)

In [18]:
model = L.Classifier(MLP(1000,1),
                     lossfun=F.mean_squared_error,
                     accfun=F.r2_score)
model.compute_accuracy = False

In [ ]:
model = (MLP(1000,1)
loss = F.mean_square_error(model(x_train),y_train)
accfun = F.r2_score

In [ ]:
# 分類（model classifier）
net = MLP(1000, 5)
model = L.Classifier(net)

In [36]:
optimizer = chainer.optimizers.SGD()
optimizer.setup(model)

In [37]:
updater = training.updaters.StandardUpdater(train_iter, optimizer)
trainer = training.Trainer(updater, (20, 'epoch'), out="Result2018_oono/%s" % time.strftime("%Y%m%d%H%M%S"))

In [38]:
# Evaluate the model with the test dataset for each epoch
trainer.extend(extensions.Evaluator(test_iter, model))

# Dump a computational graph from 'loss' variable at the first iteration
# The "main" refers to the target link of the "main" optimizer.
trainer.extend(extensions.dump_graph('main/loss'))

# Take a snapshot for each specified epoch
# frequency = args.epoch if args.frequency == -1 else max(1, args.frequency)
# trainer.extend(extensions.snapshot(), trigger=(frequency, 'epoch'))

# Write a log of evaluation statistics for each epoch
trainer.extend(extensions.LogReport())

# Save two plot images to the result dir
trainer.extend(
    extensions.PlotReport(['main/loss', 'validation/main/loss'],
                          'epoch', file_name='loss.png'))
trainer.extend(
    extensions.PlotReport(
        ['main/accuracy', 'validation/main/accuracy'],
        'epoch', file_name='accuracy.png'))

# Print selected entries of the log to stdout
# Here "main" refers to the target link of the "main" optimizer again, and
# "validation" refers to the default name of the Evaluator extension.
# Entries other than 'epoch' are reported by the Classifier link, called by
# either the updater or the evaluator.
trainer.extend(extensions.PrintReport(
    ['epoch', 'main/loss', 'validation/main/loss',
     'main/accuracy', 'validation/main/accuracy', 'elapsed_time']))

# Print a progress bar to stdout
trainer.extend(extensions.ProgressBar())

# if args.resume:
#     # Resume from a snapshot
#     chainer.serializers.load_npz(args.resume, trainer)


In [43]:
trainer.run()

Exception in main training loop: 
Invalid operation is performed in: MeanSquaredError (Forward)

Expect: x0.shape == x1.shape
Actual: (100, 0) != (100,)
Traceback (most recent call last):
  File "C:\Users\Owner\Anaconda3\lib\site-packages\chainer\training\trainer.py", line 315, in run
    update()
  File "C:\Users\Owner\Anaconda3\lib\site-packages\chainer\training\updaters\standard_updater.py", line 165, in update
    self.update_core()
  File "C:\Users\Owner\Anaconda3\lib\site-packages\chainer\training\updaters\standard_updater.py", line 177, in update_core
    optimizer.update(loss_func, *in_arrays)
  File "C:\Users\Owner\Anaconda3\lib\site-packages\chainer\optimizer.py", line 680, in update
    loss = lossfun(*args, **kwds)
  File "C:\Users\Owner\Anaconda3\lib\site-packages\chainer\link.py", line 242, in __call__
    out = forward(*args, **kwargs)
  File "C:\Users\Owner\Anaconda3\lib\site-packages\chainer\links\model\classifier.py", line 144, in forward
    self.loss = self.lossfun(

InvalidType: 
Invalid operation is performed in: MeanSquaredError (Forward)

Expect: x0.shape == x1.shape
Actual: (100, 0) != (100,)

In [2]:
import numpy
import pandas
import glob
import os
import sys
import csv

In [4]:
csv = pandas.read_csv("C:/Users/Owner/Desktop/[190611]kurasawa/Blood_glucose_level/1.csv", header=None)

f = open("C:/Users/Owner/Desktop/[190611]kurasawa/Blood_glucose_level/1.csv","r")
csv_list = []
csv_list.append(csv)

print(csv.shape)


(195609, 3)


In [5]:
# Qiita : "csvを100個読み込ませたかった" より引用
csv_search_path = r"C:/Users/Owner/Desktop/[190611]kurasawa/Blood_glucose_level/*.csv"
csv_path_ary = glob.glob(csv_search_path)

print(csv_search_path) # 配列が空だった（なぜ？）

csv[0:10]
csv2 = pandas.read_csv("C:/Users/Owner/Desktop/[190611]kurasawa/Blood_glucose_level/2.csv", header=None)
print(csv2.shape)

C:/Users/Owner/Desktop/[190611]kurasawa/Blood_glucose_level/*.csv
(213607, 3)


In [ ]:
# numpy.loadtxt でcsvが読み込めるかもしれない
# csvを読み込んで，配列の型とデータタイプの確認
csv_data = numpy.loadtxt("C:/Users/Owner/Desktop/[190611]kurasawa/Blood_glucose_level/1.csv",delimiter=",")
csv_data2 = numpy.loadtxt("C:/Users/Owner/Desktop/[190611]kurasawa/Blood_glucose_level/2.csv",delimiter=",")

def array_info(x):
    print("配列の型", x.shape)
    print("配列の要素のデータ型", x.dtype)

    if len(x) >=10:
        print("配列の中身(上から10列)\n" , x[:10], "\n")
    else:
        print("配列の中身\n", x, "\n")

array_info(csv_data)

In [ ]:
csv_list = []
for i in range():
    csv_list =


arr = numpy.vstack((csv_data,csv_data2))
arr2 = arr.T
print(arr2.shape)

In [ ]:
# コードログ(codeday.me)にあった処理
path = r'C:/Users/Owner/Desktop/[190611]kurasawa/Blood_glucose_level' #use your path
allfiles = glob.glob(path + "/1.csv")

print(path)
print(allfiles)

frame = pandas.DataFrame()
list_ = []

for file_ in allfiles:
    df = pandas.read_csv(file_, index_col=None, header=None)
    list_.append(df)

frame = pandas.concat(list_)

In [7]:
a = glob.glob("C:/Users/Owner/Desktop/[190611]kurasawa/Blood_glucose_level/1.csv")
print(a)

[]


In [18]:
b = pandas.read_csv('C:/Users/Owner/Desktop/[190611]kurasawa/Blood_glucose_level/1.csv')

# 血糖値　回帰予測

In [7]:
data, teach = easy_chainer.load_Data("C:/Users/Owner/Desktop/wrist_height_length_random.xls")
data = data.astype(numpy.float32)
teach = teach
print(teach)

[194.  97. 150. 222. 109. 165. 135. 205. 122. 172. 167. 142. 111. 160.
  93. 119. 160. 103. 133. 149. 100. 129. 219.  94.  93.  94. 232.  94.
 108. 105. 105. 188. 107. 171. 206. 167. 208. 163. 169.  97. 166. 192.
 158. 175. 171. 103. 140. 146.  94. 109. 121. 177. 159. 108. 210.  94.
 203. 104. 140. 112.]


C:\Users\Owner\Desktop\LAB\LAB 2018\jupyter File\easy_chainer.py:87: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  data = xls.as_matrix()[:-1]
C:\Users\Owner\Desktop\LAB\LAB 2018\jupyter File\easy_chainer.py:88: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  teach = xls.as_matrix()[-1]


In [9]:
print(data.shape)

(1000, 60)


In [10]:
teach = teach.astype(numpy.float32)

In [11]:
id_all = numpy.arange(1, len(teach) + 1, 1).astype(numpy.int32) - 1
print(id_all)

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58 59]


In [13]:
id_train = numpy.random.choice(id_all, 50, replace=False) #重複なし
print(id_train)

[24 28  9  0 51 26 46 23 36 18 41 20 29 52 49 56 59 35 21 10  6 44 57  3
 45 33 13 34  8 12 40 55  7 14 27 38 15  2  5 19 22 54 17 58 47 53 42 30
 11  4]


In [14]:
id_test = numpy.delete(id_all, id_train)
print(id_test)

[ 1 16 25 31 32 37 39 43 48 50]


In [15]:
x_train, y_train = data[:, id_train], teach[id_train]
x_test, y_test = data[:, id_test], teach[id_test]

In [16]:
train = tuple_dataset.TupleDataset(x_train.T, y_train.reshape(-1, ))
test = tuple_dataset.TupleDataset(x_test.T, y_test.reshape(-1, ))

train_iter = chainer.iterators.SerialIterator(train, 50)
test_iter = chainer.iterators.SerialIterator(test, 10, repeat=False, shuffle=False)

In [19]:
model = L.Classifier(MLP(1000,1),
                     lossfun=F.mean_squared_error,
                     accfun=F.r2_score)
model.compute_accuracy = False

In [20]:
optimizer = chainer.optimizers.SGD()
optimizer.setup(model)

In [21]:
updater = training.updaters.StandardUpdater(train_iter, optimizer)
trainer = training.Trainer(updater, (20, 'epoch'), out="Result2018_oono/%s" % time.strftime("%Y%m%d%H%M%S"))

In [22]:
# Evaluate the model with the test dataset for each epoch
trainer.extend(extensions.Evaluator(test_iter, model))

# Dump a computational graph from 'loss' variable at the first iteration
# The "main" refers to the target link of the "main" optimizer.
trainer.extend(extensions.dump_graph('main/loss'))

# Take a snapshot for each specified epoch
# frequency = args.epoch if args.frequency == -1 else max(1, args.frequency)
# trainer.extend(extensions.snapshot(), trigger=(frequency, 'epoch'))

# Write a log of evaluation statistics for each epoch
trainer.extend(extensions.LogReport())

# Save two plot images to the result dir
trainer.extend(
    extensions.PlotReport(['main/loss', 'validation/main/loss'],
                          'epoch', file_name='loss.png'))
trainer.extend(
    extensions.PlotReport(
        ['main/accuracy', 'validation/main/accuracy'],
        'epoch', file_name='accuracy.png'))

# Print selected entries of the log to stdout
# Here "main" refers to the target link of the "main" optimizer again, and
# "validation" refers to the default name of the Evaluator extension.
# Entries other than 'epoch' are reported by the Classifier link, called by
# either the updater or the evaluator.
trainer.extend(extensions.PrintReport(
    ['epoch', 'main/loss', 'validation/main/loss',
     'main/accuracy', 'validation/main/accuracy', 'elapsed_time']))

# Print a progress bar to stdout
trainer.extend(extensions.ProgressBar())

# if args.resume:
#     # Resume from a snapshot
#     chainer.serializers.load_npz(args.resume, trainer)


In [23]:
trainer.run()

Exception in main training loop: 
Invalid operation is performed in: MeanSquaredError (Forward)

Expect: x0.shape == x1.shape
Actual: (50, 1) != (50,)
Traceback (most recent call last):
  File "C:\Users\Owner\Anaconda3\lib\site-packages\chainer\training\trainer.py", line 315, in run
    update()
  File "C:\Users\Owner\Anaconda3\lib\site-packages\chainer\training\updaters\standard_updater.py", line 165, in update
    self.update_core()
  File "C:\Users\Owner\Anaconda3\lib\site-packages\chainer\training\updaters\standard_updater.py", line 177, in update_core
    optimizer.update(loss_func, *in_arrays)
  File "C:\Users\Owner\Anaconda3\lib\site-packages\chainer\optimizer.py", line 680, in update
    loss = lossfun(*args, **kwds)
  File "C:\Users\Owner\Anaconda3\lib\site-packages\chainer\link.py", line 242, in __call__
    out = forward(*args, **kwargs)
  File "C:\Users\Owner\Anaconda3\lib\site-packages\chainer\links\model\classifier.py", line 144, in forward
    self.loss = self.lossfun(se

InvalidType: 
Invalid operation is performed in: MeanSquaredError (Forward)

Expect: x0.shape == x1.shape
Actual: (50, 1) != (50,)